In [ ]:
import re
import os

min_p = []
min_r = []
min_f = []
max_p = []
max_r = []
max_f = []
sred_p = []
sred_r = []
sred_f = []
name = []
pr = []
for subdir, _, files in os.walk('/content/'):
    for file in files:

        if file.endswith(".txt"):
            file_path = os.path.join(subdir, file)
            with open(file_path, "r", encoding="utf-8", errors='replace') as f:
                #print(file_path)
                precisions = []
                recalls = []
                f1s = []
                content = f.read()
                precision_matches = re.findall(r'BERTScore Precision:\s*([0-9.]+)', content)
                recall_matches = re.findall(r'BERTScore Recall:\s*([0-9.]+)', content)
                f1_matches = re.findall(r'BERTScore F1:\s*([0-9.]+)', content)

                precisions = [float(p) for p in recall_matches if float(p) != 0]
                pr.append(precisions)
                name.append(file_path)
                print(max(precisions))
                print(file_path)
                print()
                #recalls = [float(r) for r in recall_matches if float(r) != 0]
                #f1s = [float(f) for f in f1_matches if float(f) != 0]

                #min_p.append(min(precisions))
                #max_p.append(max(precisions))
                #min_r.append(min(recalls))
                #max_r.append(max(recalls))
                #min_f.append(min(f1s))
                #max_f.append(max(f1s))
                #sred_p += precisions
                #sred_r += recalls
                #sred_f += f1s
                # Вывод средних значений
                #print(f"Средний BERTScore Precision: {sum(precisions)/len(precisions):.4f}")
                #print(f"Средний BERTScore Recall:    {sum(recalls)/len(recalls):.4f}")
                #print(f"Средний BERTScore F1:        {sum(f1s)/len(f1s):.4f}")
                #print()
                #print()
print(sorted([(max(p), n) for p, n in zip(pr, name)], reverse=True)[:10])
#print(f"{sum(min_p) / len(min_p):.4f}")
#print(f"{sum(max_p) / len(max_p):.4f}")
#print()
#print(f"{sum(min_r) / len(min_r):.4f}")
#print(f"{sum(max_r) / len(max_r):.4f}")
#print()
#print(f"{sum(min_f) / len(min_f):.4f}")
#print(f"{sum(max_f) / len(max_f):.4f}")

#print(f"{min(min_p):.4f}")
#print(f"{max(max_p):.4f}")
#print()
#print(f"{min(min_r):.4f}")
#print(f"{max(max_r):.4f}")
#print()
#print(f"{min(min_f):.4f}")
#print(f"{max(max_f):.4f}")

#print()
#print(f"{sum(sred_p) / len(sred_p):.4f}")
#print()
#print(f"{sum(sred_r) / len(sred_r):.4f}")
#print()
#print(f"{sum(sred_f) / len(sred_f):.4f}")

0.7499
/content/Летние Олимпийские игры 2020.txt

0.6607
/content/Большой адронный коллайдер.txt

0.8443
/content/Искусственный интеллект.txt

0.6757
/content/Dota 2.txt

0.8588
/content/Сколково (инновационный центр).txt

0.7393
/content/Геморрагическая лихорадка Эбола.txt

0.6585
/content/COVID-19.txt

0.5915
/content/C++.txt

0.9685
/content/Чёрная дыра.txt

0.5792
/content/YouTube.txt

0.7732
/content/Иннополис.txt

0.5361
/content/Чёрная смерть.txt

0.5671
/content/Tomb Raider (игра, 2013).txt

0.8452
/content/Присоединение Крыма к Российской Федерации.txt

0.7835
/content/Применение искусственного интеллекта.txt

0.6519
/content/TikTok.txt

0.7426
/content/Марс.txt

0.7111
/content/Квантовый компьютер.txt

0.8394
/content/РИА Новости.txt

0.6373
/content/Portal 2.txt

0.7269
/content/Солнечная система.txt

0.6963
/content/Экономика США.txt

0.7661
/content/Яндекс.txt

0.8647
/content/Летние Олимпийские игры 2024.txt

0.683
/content/Вирус иммунодефицита человека.txt

0.8556
/conte

In [ ]:
import numpy as np
from scipy.stats import bootstrap

def calc(data):
  data = np.array(data)
  data1 = (data,)
  bootstrap_ci = bootstrap(data1, np.mean, confidence_level=0.95, n_resamples=len(data)-1)

  dist = bootstrap_ci.bootstrap_distribution
  mean = np.quantile(dist, q=0.5)
  min = np.quantile(dist, q=0.025)
  max = np.quantile(dist, q=0.975)
  print(mean)
  print(min)
  print(max)
  print()

calc(sred_p)
calc(sred_r)
calc(sred_f)

0.5334563679245283
0.5267862735849056
0.5395877830188679

0.5444884433962264
0.535396733490566
0.5532665683962263

0.5355603773584905
0.5293551061320755
0.5424484787735849



In [ ]:
from bs4 import BeautifulSoup
import requests

html = requests.get('https://www.britannica.com/topic/European-Union', headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0 Safari/537.36'
}).text
soup = BeautifulSoup(html, 'html.parser')
text = soup.get_text(strip=True, separator=' ')
text

"European Union (EU) | Definition, Flag, Purpose, History, & Members | Britannica Search Britannica Click here to search Search Britannica Click here to search SUBSCRIBE SUBSCRIBE Login https://premium.britannica.com/premium-membership/?utm_source=premium&utm_medium=nav-login-box&utm_campaign=evergreen SUBSCRIBE Home History & Society Science & Tech Biographies Animals & Nature Geography & Travel Arts & Culture ProCon Money Games & Quizzes Videos On This Day One Good Fact Dictionary New Articles History & Society Lifestyles & Social Issues Philosophy & Religion Politics, Law & Government World History Science & Tech Health & Medicine Science Technology Biographies Browse Biographies Animals & Nature Birds, Reptiles & Other Vertebrates Bugs, Mollusks & Other Invertebrates Environment Fossils & Geologic Time Mammals Plants Geography & Travel Geography & Travel Arts & Culture Entertainment & Pop Culture Literature Sports & Recreation Visual Arts Companions Demystified Image Galleries List

In [ ]:
s = '''
Пригожин, Евгений Викторович	29 800,4
Вторжение России на Украину (с 2022)	23 143,2
ЧВК «Вагнер»	23 035,5
Список умерших в 2023 году	16 607,6
Путин, Владимир Владимирович	16 492,9
Россия	16 096,6
Слово пацана. Кровь на асфальте	13 521,0
Горшенёв, Михаил Юрьевич	13 289,2
Государство Палестина	11 482,7
Король и Шут	10 812,3
Сектор Газа	10 806,9
Уткин, Дмитрий Валерьевич	10 666,7
Оппенгеймер, Роберт	10 470,4
Украина	10 234,0
Израиль	9 663,9
Бахмут	8 327,7
Сталин, Иосиф Виссарионович	8 179,8
Союз Советских Социалистических Республик	8 177,9
Президентские выборы в России (2024)	8 100,1
ChatGPT	7 869,8
Головкин, Сергей Александрович (серийный убийца)	7 649,6
Соединённые Штаты Америки	7 612,7
Зеленский, Владимир Александрович	7 398,9
Екатерина II	7 095,8
Список фильмов кинематографической вселенной Marvel	6 854,4
Криштиану Роналду	6 851,7
Москва	6 846,4
Гитлер, Адольф	6 817,4
Первая мировая война	6 749,3
Atomic Heart	6 528,5
Чикатило, Андрей Романович	6 504,4
Потери сторон в период вторжения России на Украину	6 465,9
ХАМАС	6 359,1
Вторая мировая война	6 282,5
Костомаров, Роман Сергеевич	6 254,2
Пушкин, Александр Сергеевич	6 242,4
Санкт-Петербург	6 071,0
Месси, Лионель	6 014,9
Shaman	5 868,8
Оппенгеймер (фильм)	5 685,6
Мятеж ЧВК «Вагнер»	5 603,5
Шойгу, Сергей Кужугетович	5 551,1
Пётр I	5 509,2
Список государств и зависимых территорий по населению	5 445,8
Знаки зодиака	5 345,0
Ленин, Владимир Ильич	5 290,3
Российская империя	5 245,3
Бои за Бахмут	5 234,3
Российско-украинская война	5 220,6
Китай	5 167,5
YouTube	5 121,6
Гарри Поттер (серия фильмов)	5 068,8
Пересильд, Юлия Сергеевна	5 036,9
Казахстан	5 020,0
Аватар: Путь воды	4 979,6
Янковский, Иван Филиппович	4 952,8
Князев, Андрей Сергеевич	4 879,0
250 лучших фильмов по версии IMDb	4 831,2
Английский язык	4 807,7
НАТО	4 696,7
Елизавета II	4 691,0
Кадыров, Рамзан Ахматович	4 652,6
Чурикова, Инна Михайловна	4 610,8
Сталин, Василий Иосифович	4 590,7
Пугачёва, Алла Борисовна	4 587,5
Субъекты Российской Федерации	4 580,6
Бандера, Степан Андреевич	4 568,9
Маск, Илон	4 454,8
Белоруссия	4 432,9
Атомные бомбардировки Хиросимы и Нагасаки	4 304,8
Великая Отечественная война	4 264,4
Арабо-израильский конфликт	4 263,6
Великобритания	4 192,0
Германия	4 179,9
Байден, Джо	4 167,3
Цой, Виктор Робертович	4 164,1
Наполеон I	4 152,4
Авария на Чернобыльской АЭС	4 150,8
Одни из нас (телесериал)	4 119,4
Павел I	4 113,6
Карл III (король Великобритании)	4 091,0
БРИКС	4 013,1
Раневская, Фаина Георгиевна	3 985,5
Ди Каприо, Леонардо	3 975,2
Call of Duty (серия игр)	3 968,4
Орден «За заслуги перед Отечеством»	3 968,4
Война в Донбассе (2014—2022)	3 961,7
Кинжал (авиационный ракетный комплекс)	3 954,2
Стейтем, Джейсон	3 953,7
Стрелков, Игорь Иванович	3 951,9
Леопард-2	3 939,0
Чебурашка (фильм)	3 907,2
Уэнздей	3 883,9
Международный женский день	3 878,5
Википедия	3 870,9
Корейская Народно-Демократическая Республика	3 828,8
Шаляпин, Фёдор Иванович	3 820,3
Землетрясение в Турции и Сирии (2023)	3 811,4
Николай II	3 796,3
Кемстач, Леон Ильич	3 794,6
Гагарин, Юрий Алексеевич	3 775,1
Нацистская Германия	3 751,7
Иван Грозный	3 745,4
Отче наш	3 703,8
Палестино-израильский конфликт	3 688,9
Европейский союз	3 674,0
Мизулина, Екатерина Михайловна	3 668,5
Террористический акт в Беслане	3 667,4
Список президентов США	3 667,0
Дюбарри, Жанна	3 665,3
Список государств и зависимых территорий по площади	3 663,6
Первая чеченская война	3 663,4
Приднестровская Молдавская Республика	3 646,4
Титаник	3 642,8
Казанский феномен	3 630,9
Уиллис, Брюс	3 628,2
Герасимов, Валерий Васильевич	3 584,7
Вызов (фильм, 2023)	3 574,9
Афганская война (1979—1989)	3 569,1
Семь смертных грехов	3 566,5
Террористические акты 11 сентября 2001 года	3 564,5
Способы самоубийства	3 559,5
Иран	3 554,8
Турция	3 542,6
Юдашкин, Валентин Абрамович	3 531,5
Брежнев, Леонид Ильич	3 529,9
Барби (фильм)	3 520,1
Киев	3 518,6
Иисус Христос	3 512,5
Нагорный Карабах	3 511,5
Азербайджан	3 510,1
Толстой, Лев Николаевич	3 509,6
Фишер (сериал)	3 497,2
Хрущёв, Никита Сергеевич	3 442,9
Якубович, Леонид Аркадьевич	3 437,3
Индия	3 434,3
Ивлеева, Настя	3 396,0
Горбачёв, Михаил Сергеевич	3 394,8
Марцинкевич, Максим Сергеевич	3 391,5
Гослинг, Райан	3 374,7
Залужный, Валерий Фёдорович	3 367,7
Лукашенко, Александр Григорьевич	3 352,7
Звёздные войны	3 332,0
Джон Уик 4	3 327,8
Гоголь, Николай Васильевич	3 321,0
2023 год	3 315,9
Форсаж 10	3 312,8
Список руководителей СССР	3 310,7
Список самых кассовых фильмов	3 291,0
Есенин, Сергей Александрович	3 288,9
Хасбик	3 262,5
Высоцкий, Владимир Семёнович	3 258,3
Ривз, Киану	3 252,7
Перри, Мэттью (актёр)	3 245,5
Сталинградская битва	3 241,9
Искандер (ракетный комплекс)	3 235,4
Stray Kids	3 234,0
День святого Валентина	3 209,3
Прилепин, Захар	3 207,7
Война во Вьетнаме	3 202,0
The Last of Us	3 200,1
Чан, Джеки	3 197,1
Пересильд, Анна Алексеевна	3 196,5
Острые козырьки	3 194,1
Пётр III	3 191,5
General Dynamics F-16 Fighting Falcon	3 188,3
Распад СССР	3 180,4
Михалков, Никита Сергеевич	3 179,3
Су-57	3 177,7
Грузия	3 171,2
Османская империя	3 158,1
Польша	3 155,1
Депп, Джонни	3 148,0
Япония	3 146,4
Ислам	3 145,6
Ортега, Дженна	3 140,5
Кабаева, Алина Маратовна	3 139,4
Содружество Независимых Государств	3 139,2
Игра престолов (телесериал)	3 136,5
Смольянинов, Артур Сергеевич	3 136,3
Telegram	3 134,6
Медведев, Дмитрий Анатольевич	3 131,8
Объединённые Арабские Эмираты	3 129,3
Шварценеггер, Арнольд	3 115,8
Вооружённые силы Российской Федерации	3 109,0
Екатеринбург	3 105,5
Минекаев, Рузиль Рамилевич	3 099,7
Суровикин, Сергей Владимирович	3 099,4
Ельцин, Борис Николаевич	3 079,9
Канада	3 079,9
Франция	3 065,7
Король и Шут (сериал)	3 061,6
Армения	3 050,6
Узбекистан	3 047,9
Фашизм	3 041,4
Мухаммед	3 041,0
Дюмин, Алексей Геннадьевич	3 040,5
MrBeast	3 038,7
Minecraft	3 032,1
Стражи Галактики. Часть 3	3 029,7
Калининград	3 026,9
Вторая чеченская война	3 017,3
Коды субъектов Российской Федерации	3 011,2
ВКонтакте	3 009,6
Крым	2 994,9
Диана, принцесса Уэльская	2 988,0
Мэрилин Монро	2 987,4
Иудаизм	2 985,5
Киргизия	2 985,4
Гражданская война в России	2 980,9
Александр II	2 975,7
Население России	2 971,6
Список государств — членов НАТО	2 967,5
Капибара	2 965,0
Жуков, Георгий Константинович	2 951,2
Русский язык	2 945,2
M1 «Абрамс»	2 944,0
Синдром дефицита внимания и гиперактивности	2 943,1
Александр I	2 940,3
Буданов, Кирилл Алексеевич	2 938,5
Х-22	2 934,6
Т-90	2 931,0
Республика Корея	2 924,3
Афганистан	2 919,9
Реал Мадрид (футбольный клуб)	2 918,0
Зайцев, Вячеслав Михайлович	2 917,1
Берия, Лаврентий Павлович	2 909,9
Китайская Республика (Тайвань)	2 907,5
Питт, Брэд	2 904,0
Мариуполь	2 901,5
Эйнштейн, Альберт	2 900,1
Список городов России	2 888,1
BTS	2 888,0
Вторжение ХАМАС в Израиль (2023)	2 883,8
Достоевский, Фёдор Михайлович	2 871,9
Евровидение-2023	2 870,5
Навальный, Алексей Анатольевич	2 864,5
Президентские выборы в США (2024)	2 860,9
Троцкий, Лев Давидович	2 857,2
Флаг России	2 854,3
Джексон, Майкл	2 851,6
Киевская Русь	2 848,9
Владлен Татарский	2 845,7
XXXTentacion	2 838,5
Во все тяжкие	2 835,0
Корчевников, Борис Вячеславович	2 831,8
Елизавета Петровна	2 830,9
Евреи	2 809,9
С-300	2 805,7
Асти, Анна	2 797,6
Блокада Ленинграда	2 796,5
Ломоносов, Михаил Васильевич	2 793,0
Николай I	2 784,8
Кока, Клава	2 784,1
Герой Российской Федерации	2 777,5
Обсессивно-компульсивное расстройство	2 774,3
Эминем	2 773,3
Нижний Новгород	2 772,1
Лермонтов, Михаил Юрьевич	2 771,8
Нагорно-Карабахская Республика	2 767,9
Советско-финляндская война (1939—1940)	2 762,0
Жириновский, Владимир Вольфович	2 752,0
Хади Такташ (организованная преступная группировка)	2 750,2
Кошка	2 750,2
Швейцария	2 748,1
Киркоров, Филипп Бедросович	2 743,6
Робби, Марго	2 737,1
Эскобар, Пабло	2 733,1
Круз, Том	2 725,9
Корь	2 721,3
Американская история ужасов	2 719,5
Спитакское землетрясение	2 717,0
Национальный состав России	2 715,8
Шатунов, Юрий Васильевич	2 715,4
Добронравов, Фёдор Викторович	2 707,8
Солнечная система	2 692,5
Молдавия	2 684,4
Список стран по ВВП (ППС) на душу населения	2 682,9
Донецкая Народная Республика	2 673,3
Автомат Калашникова	2 664,4
Александр Македонский	2 663,0
Жора Крыжовников	2 653,9
Прилучный, Павел Валерьевич	2 652,6
Австралия	2 645,8
Трамп, Дональд	2 640,9
Ким Чен Ын	2 638,6
Список игроков НХЛ, забросивших 500 и более шайб	2 635,1
Сингапур	2 633,3
Нагиев, Дмитрий Владимирович	2 628,8
Чехов, Антон Павлович	2 626,5
Пи (число)	2 625,9
Отечественная война 1812 года	2 621,3
Янковский, Олег Иванович	2 619,1
Список стран по численности вооружённых сил и военизированных формирований	2 614,2
Организация Объединённых Наций	2 611,3
Донецк	2 606,8
Хороший доктор (телесериал, 2017)	2 605,2
Горин, Геннадий Александрович	2 599,4
Соловьёв, Владимир Рудольфович	2 576,9
Барселона (футбольный клуб)	2 575,3
Александр III	2 574,9
Европа	2 574,9
Мирошниченко, Ирина Петровна	2 571,4
Битва экстрасенсов	2 568,8
Нидерланды	2 562,1
Атака на титанов	2 560,4
Мавроди, Сергей Пантелеевич	2 557,2
Саудовская Аравия	2 557,1
Сталлоне, Сильвестр	2 556,9
14/88	2 553,2
Дагестан	2 551,2
Сулейман I	2 549,7
Список стран по ВВП (ППС)	2 548,4
Пираты Карибского моря (серия фильмов)	2 544,0
Мерфи, Киллиан	2 540,2
Заворотнюк, Анастасия Юрьевна	2 529,7
Ту-95	2 528,4
Нурмагомедов, Хабиб Абдулманапович	2 527,3
Джоли, Анджелина	2 519,5
Мбаппе, Килиан	2 512,7
Десять заповедей	2 501,1
Аргентина	2 500,4
Безруков, Сергей Витальевич	2 498,8
Вооружённые силы Украины	2 498,7
Убийцы цветочной луны	2 496,9
Византия	2 494,7
Су-35	2 491,6
Фредди Меркьюри	2 482,9
Чайковский, Пётр Ильич	2 482,7
Уральские пельмени	2 482,3
Збарская, Регина Николаевна	2 479,0
Вампиры средней полосы	2 473,4
Аватар (фильм, 2009)	2 469,5
Кейдж, Николас	2 466,9
Абхазия	2 465,2
Дамер, Джеффри	2 464,0
Иерусалим	2 463,8
Т-72	2 460,8
Петров, Александр Андреевич (актёр)	2 459,5
Бурунов, Сергей Александрович	2 458,2
Холанн, Эрлинг	2 457,8
Ибрагимович, Златан	2 451,4
Тяжёлая огнемётная система	2 448,6
Басаев, Шамиль Салманович	2 447,8
Масонство	2 444,7
Италия	2 441,6
Овечкин, Александр Михайлович	2 441,3
Волгоград	2 435,4
Ефремов, Михаил Олегович	2 433,8
Маяковский, Владимир Владимирович	2 428,2
Блиновская, Елена Олеговна	2 427,2
Манчестер Сити	2 426,4
Т-14	2 416,8
Список войн и вооружённых конфликтов России	2 414,1
Кончаловский, Андрей Сергеевич	2 412,0
Хезболла	2 408,2
Аннексия Крыма Российской Федерацией	2 400,9
Ричи, Гай	2 399,3
БДСМ	2 398,0
Русский алфавит	2 393,8
Краснодар	2 391,6
Лига чемпионов УЕФА	2 381,4
Южно-Африканская Республика	2 380,9
Арестович, Алексей Николаевич	2 380,8
Казань	2 377,0
Русский добровольческий корпус	2 373,8
Назаров, Дмитрий Юрьевич	2 372,2
Биполярное расстройство	2 367,4
Список государств	2 363,8
Нюрнбергский процесс	2 354,3
Хэллоуин	2 352,8
Неймар	2 352,1
Саддам Хусейн	2 351,5
Pornhub	2 351,2
Орден Ленина	2 349,8
Ливан	2 348,5
Яндекс	2 346,7
Исламское государство	2 344,8
Ростов-на-Дону	2 343,5
Ан-Наср (футбольный клуб, Эр-Рияд)	2 340,3
МиГ-29	2 336,8
Тинькофф банк	2 333,5
Горшенёв, Алексей Юрьевич	2 330,0
Война в Грузии (2008)	2 328,0
Буддизм	2 325,5
Де Армас, Ана	2 324,0
Владимир Святославич	2 318,9
Египет	2 318,1
Новосибирск	2 317,5
Население Земли	2 302,0
Мишустин, Михаил Владимирович	2 301,9
Белгород	2 299,4
Кухня (телесериал)	2 294,5
Испания	2 286,6
Куколд	2 286,0
Список вооружения и военной техники Сухопутных войск Российской Федерации	2 281,6
Всё везде и сразу	2 280,6
Римская империя	2 276,2
Периодическая система химических элементов	2 274,4
Нью-Йорк	2 273,8
Нигер	2 273,3
ДШРГ «Русич»	2 266,5
Булгаков, Михаил Афанасьевич	2 265,1
Финляндия	2 263,7
МиГ-31	2 263,2
Ядерное оружие	2 260,6
Инстасамка	2 259,8
Пресли, Элвис	2 258,4
Югославия	2 256,7
Пэтриот	2 253,6
Сербия	2 249,3
Нацизм	2 247,3
Астана	2 246,6
Латинский язык	2 244,0
Христианство	2 240,2
Орден Мужества	2 239,9
Челентано, Адриано	2 239,5
Миронов, Андрей Александрович	2 237,4
Свастика	2 236,6
MILF	2 236,1
Романовы	2 227,5
Попков, Михаил Викторович	2 226,8
Кикабидзе, Вахтанг Константинович	2 225,1
Балабанов, Алексей Октябринович	2 223,6
Лавров, Сергей Викторович	2 223,4
Эрдоган, Реджеп Тайип	2 215,1
Российская Советская Федеративная Социалистическая Республика	2 214,6
Правители Российского государства	2 212,9
Ту-22М	2 210,4
Чингисхан	2 210,1
Норвегия	2 209,8
Медаль ордена «За заслуги перед Отечеством»	2 209,4
Шейк, Ирина	2 206,3
Черчилль, Уинстон	2 205,8
Гонконг	2 201,4
Распутин, Григорий Ефимович	2 201,4
Тайсон, Майк	2 199,3
Storm Shadow	2 196,6
Дуров, Павел Валерьевич	2 195,5
Бразилия	2 192,7
Джонсон, Дуэйн	2 192,6
Размер полового члена человека	2 186,7
Чёрное зеркало (телесериал)	2 184,8
Максим Горький	2 180,9
Бригада (телесериал)	2 179,9
Чеботина, Люся	2 174,7
Летов, Егор	2 171,2
Каховская ГЭС	2 171,1
Крещение Руси	2 170,5
Крушение «Титаника»	2 169,7
Президент Российской Федерации	2 169,6
Аксёнова, Любовь Павловна	2 167,8
Моргенштерн	2 164,7
Йемен	2 163,5
Мандалорец	2 161,5
Али, Мухаммед	2 160,8
Президентские выборы в Турции (2023)	2 159,8
События сентября — октября 1993 года в Москве	2 157,9
Круг, Михаил Владимирович	2 156,8
Бодров, Сергей Сергеевич	2 154,6
Резня в Буче	2 152,9
Список иностранных агентов (Россия)	2 151,3
Blackpink	2 151,2
Крымская война	2 148,6
Очень странные дела	2 146,2
Папины дочки	2 145,4
Пеле	2 142,9
Рейнольдс, Райан	2 140,2
Кинематографическая вселенная Marvel	2 139,4
Харди, Том	2 136,2
Су-27	2 130,9
Московский метрополитен	2 128,8
Сваты	2 128,8
Пелевин, Виктор Олегович	2 119,4
Кассетный боеприпас	2 117,0
Бикович, Милош	2 115,7
Война Судного дня	2 106,3
Холокост	2 105,7
Список стран по ВВП (номинал)	2 104,9
Лебедев, Артемий Андреевич	2 104,1
Махачев, Ислам Рамазанович	2 098,4
Пахмутова, Александра Николаевна	2 097,1
Джилленхол, Джейк	2 097,0
Аутизм	2 096,9
Лопес, Дженнифер	2 096,8
Анальный секс	2 096,2
АК-74	2 095,3
Земля	2 095,2
Екатерина I	2 091,0
Переводчик (фильм)	2 083,3
Синдром Аспергера	2 082,7
Минет	2 081,0
Волынская резня	2 079,8
Армия обороны Израиля	2 078,2
Ева Эльфи	2 078,1
Алма-Ата	2 077,7
Пасха	2 077,5
Лоуренс, Дженнифер	2 076,3
Черногория	2 074,1
Гай Юлий Цезарь	2 071,6
Байкал	2 071,1
Чёрный дельфин	2 069,3
Создание Государства Израиль	2 065,9
Мажор (телесериал)	2 064,1
Нейронная сеть	2 063,6
История Украины	2 061,7
Кеннеди, Джон Фицджералд	2 061,7
Фрейзер, Брендан	2 061,4
Миядзаки, Хаяо	2 061,3
Беллуччи, Моника	2 061,2
Монголия	2 060,2
Александр Ярославич Невский	2 059,8
Талибан	2 059,8
Джобс, Стив	2 058,9
Янковский, Филипп Олегович	2 057,2
Посттравматическое стрессовое расстройство	2 055,7
Речь Посполитая	2 054,9
Пакистан	2 054,9
Людовик XV	2 053,9
Пацаны (телесериал)	2 052,5
Венгрия	2 051,3
Доллар США	2 051,3
Половой акт	2 048,8
Евромайдан	2 047,5
Владивосток	2 046,0
Таиланд	2 041,6
Стамбул	2 038,5
Т-34	2 036,7
Список городов России с населением более 100 тысяч жителей	2 034,2
Борисов, Юрий Александрович	2 030,7
Иордания	2 030,6
Grand Theft Auto V	2 029,7
Половой член человека	2 028,4
Бангладеш	2 026,1
Челси (футбольный клуб)	2 025,2
Туркменистан	2 025,1
Роксолана	2 023,4
Трошев, Андрей Николаевич	2 020,3
Шизофрения	2 020,2
Сектор Газа (группа)	2 019,3
Суворов, Александр Васильевич	2 014,1
Т-90АМ	2 014,1
Таджикистан	2 013,8
Великая французская революция	2 010,7
Газлайтинг	2 008,4
Де Ниро, Роберт	2 008,1
Искусственный интеллект	2 007,7
Lamborghini	2 005,4
Список эпизодов телесериала «Чёрное зеркало»	2 005,0
Тарантино, Квентин	2 004,8
Мефедрон	2 002,4
Рахманинов, Сергей Васильевич	2 000,8
Одесса	1 999,6
Калибр (ракета)	1 998,0
Ефремов, Никита Михайлович	1 992,6
F-35	1 991,2
Учитель, Алексей Ефимович	1 991,2
Газа	1 989,3
Усик, Александр Александрович	1 986,4
Днепр (город)	1 985,6
Кучера, Оскар Александрович	1 984,3
Землетрясение	1 984,0
Кара-Мурза, Владимир Владимирович	1 983,9
Нолан, Кристофер	1 983,8
Георг VI	1 982,9
Ходячие мертвецы (телесериал)	1 978,9
Солдаты (телесериал)	1 976,7
Си Цзиньпин	1 975,6
Магомаев, Муслим Магометович	1 973,9
Александрова, Марина Андреевна	1 970,9
Тарасова, Дарья-Аглая Викторовна	1 970,4
С-400	1 966,9
Карабахский конфликт	1 965,1
Галкин, Максим Александрович	1 964,5
Русские	1 962,3
Абдулов, Александр Гавриилович	1 957,0
Рождество Христово	1 955,3
Усама бен Ладен	1 953,0
Тесла, Никола	1 949,9
Колчак, Александр Васильевич	1 948,7
Три богатыря (франшиза)	1 947,0
Кива, Илья Владимирович	1 944,9
Су-34	1 944,1
Дудь, Юрий Александрович	1 943,9
Макконахи, Мэттью	1 942,3
Список крупнейших землетрясений	1 942,0
Машков, Владимир Львович	1 941,4
Собчак, Ксения Анатольевна	1 941,2
Индонезия	1 939,1
Сайрус, Майли	1 938,7
Бейл, Кристиан	1 938,4
1984 (роман)	1 938,4
Баста (музыкант)	1 936,5
Федеральная служба безопасности	1 935,7
Швеция	1 935,1
Аффлек, Бен	1 934,8
Wildberries	1 934,5
Ту-160	1 934,4
Ирландия	1 933,5
Октябрьская революция	1 933,3
Дудаев, Джохар Мусаевич	1 926,2
Титаник (фильм, 1997)	1 924,6
Фрейндлих, Алиса Бруновна	1 923,4
Гибель тургруппы Дятлова	1 917,5
Севастополь	1 916,7
Хронология вторжения России на Украину (с 2022)	1 914,7
Бомбардировки Югославии (1999)	1 912,6
Ка-52	1 911,8
Священная Римская империя	1 910,9
Боярский, Михаил Сергеевич	1 910,4
Ведьмак (телесериал, 2019)	1 907,4
Конюхов, Фёдор Филиппович	1 906,6
Гришечкин, Вячеслав Германович	1 905,6
Гагарина, Полина Сергеевна	1 900,9
Холодная война	1 899,0
Корейская война	1 898,1
Дислексия	1 897,2
Бумажный дом (телесериал)	1 896,1
Кобейн, Курт	1 894,2
Дом Дракона	1 891,9
Клинских, Юрий Николаевич	1 891,7
СС	1 885,5
Гайдай, Леонид Иович	1 884,6
Смит, Уилл	1 883,7
Панфилов, Глеб Анатольевич	1 883,4
Панцирь-С1	1 883,3
Самара	1 882,8
Милявская, Лолита Марковна	1 882,1
Леджер, Хит	1 881,9
Епифанцев, Владимир Георгиевич	1 881,0
Харьков	1 879,1
Куликовская битва	1 879,0
Таблица математических символов	1 876,0
Джордан, Майкл	1 874,2
Сверхъестественное (телесериал)	1 873,8
Мир! Дружба! Жвачка!	1 873,7
Ремарк, Эрих Мария	1 872,6
Муссолини, Бенито	1 871,6
Киссинджер, Генри	1 870,9
Иракская война	1 870,5
Пускепалис, Сергей Витауто	1 870,4
Список старейших людей в мире	1 870,2
Шарлот (музыкант)	1 868,8
Тина Тёрнер	1 868,4
Че Гевара, Эрнесто	1 868,3
Маска (телешоу)	1 867,5
Румыния	1 867,2
Джомолунгма	1 862,2
Теракт на Дубровке	1 859,7
с в о	1 859,0
Харламов, Гарик	1 858,7
Реальные пацаны	1 858,2
Клуб 27	1 857,3
Воля, Павел Алексеевич	1 854,0
Чечня	1 853,6
M142 HIMARS	1 849,8
Башкортостан	1 848,0
Емельяненко, Фёдор Владимирович	1 847,3
Пансексуальность	1 847,2
Список императоров России	1 845,8
Ядерный клуб	1 845,0
Кунис, Мила	1 844,6
Великий Новгород	1 842,7
Гражданская война в Сирии	1 841,5
Купер, Брэдли	1 839,7
Васильева, Вера Кузьминична	1 839,0
Hogwarts Legacy	1 838,9
12-часовой формат времени	1 834,6
Германская Демократическая Республика	1 830,5
Леонардо да Винчи	1 828,1
ZALA Ланцет	1 827,1
Русско-японская война	1 825,9
Хабенский, Константин Юрьевич	1 824,8
День защитника Отечества	1 821,8
Добронравов, Николай Николаевич	1 820,7
Сибирская язва	1 819,3
Моцарт, Вольфганг Амадей	1 818,8
Шумахер, Михаэль	1 816,7
Королёв, Сергей Павлович	1 816,4
Энистон, Дженнифер	1 815,2
Ли, Брюс	1 814,5
Соледар	1 813,2
(G)I-dle	1 813,0
Постучись в мою дверь	1 812,2
Сирия	1 810,2
Константинополь	1 809,6
Августовский путч	1 809,6
Республика Кипр	1 809,4
Калашников, Михаил Тимофеевич	1 809,4
Царь-бомба	1 809,3
Карелия	1 807,8
Гражданская война в США	1 803,6
Влагалище человека	1 801,7
Республика в составе Российской Федерации	1 801,6
Лоботомия	1 800,6
Шестидневная война	1 798,4
Крушение батискафа «Титан»	1 797,1
Певчих, Мария Константиновна	1 797,0
Бригада «Азов»	1 796,8
Оргазм	1 795,9
Панин, Алексей Вячеславович	1 794,2
Татарстан	1 793,4
Каддафи, Муаммар	1 793,3
Вавилон	1 792,3
Симоньян, Маргарита Симоновна	1 791,2
Кологривый, Никита Сергеевич	1 790,9
Виктория (королева Великобритании)	1 790,3
Семья Владимира Путина	1 789,1
Троянова, Яна Александровна	1 789,0
Т-80	1 786,8
Солнце	1 786,2
Песков, Дмитрий Сергеевич	1 784,6
Республика Косово	1 783,2
Список персонажей серии романов о Гарри Поттере	1 783,1
Леопард 1	1 782,8
Сонный паралич	1 780,7
Five Nights at Freddy’s	1 779,1
Рамадан	1 778,2
Вьетнам	1 776,8
Мастурбация	1 775,3
Ленд-лиз	1 775,0
2023 год в кино	1 773,6
Кит (фильм, 2022)	1 772,8
Человек-муравей и Оса: Квантомания	1 771,4
Единый день голосования 10 сентября 2023 года	1 770,4
Евразийский экономический союз	1 767,0
Каспаров, Гарри Кимович	1 766,4
Йовович, Милла	1 765,1
Дзюба, Артём Сергеевич	1 764,2
Кирилл (патриарх Московский)	1 762,7
Пять ночей с Фредди (фильм)	1 761,7
Курская битва	1 759,8
Чебурашка	1 755,9
Каспийское море	1 755,1
Золотая Орда	1 754,8
Газманов, Олег Михайлович	1 753,6
Йеллоустон (телесериал)	1 752,7
Люцифер	1 752,0
Республика Абхазия	1 751,5
Rammstein	1 750,3
День матери	1 750,1
Сионизм	1 748,8
Мельникова, Дарья Алексеевна	1 746,9
Шри-Ланка	1 746,8
Либерализм	1 746,2
Интер Майами	1 746,1
Абьюзивные отношения	1 744,7
Четыре всадника Апокалипсиса	1 744,6
Чемпионат Европы по футболу 2024	1 744,4
Список президентов Украины	1 741,9
Украинская Советская Социалистическая Республика	1 741,2
Ил-76	1 741,2
Паскаль, Педро	1 740,8
Вин Дизель	1 740,3
Славяне	1 740,3
Поколение Z	1 740,2
Кот в сапогах 2: Последнее желание	1 739,1
Международный фонетический алфавит	1 737,7
Патрушев, Николай Платонович	1 737,1
Люксембург	1 736,9
Делон, Ален	1 736,5
Ватикан	1 733,6
Тургенев, Иван Сергеевич	1 732,1
Ван Дамм, Жан-Клод	1 729,6
Красноярск	1 727,1
Мулявин, Владимир Георгиевич	1 726,5
Человек разумный	1 725,4
Рэдклифф, Дэниел	1 723,8
Великолепный век	1 722,3
Козловский, Данила Валерьевич	1 721,7
Ведьмак (серия романов)	1 720,0
Освенцим	1 718,5
Лебедь, Александр Иванович	1 717,5
Нисон, Лиам	1 716,6
Одноклассники (социальная сеть)	1 716,1
Пограничное расстройство личности	1 716,0
Аль-Каида	1 715,3
Валерия (певица)	1 714,9
Ирак	1 713,6
Волкановски, Александр	1 713,6
Орден Отечественной войны	1 713,5
Яшин, Лев Иванович	1 713,0
Марокко	1 711,9
Буллок, Сандра	1 709,2
Манчестер Юнайтед	1 708,3
Пари Сен-Жермен	1 708,2
Фьюри, Тайсон	1 708,0
Крид, Егор	1 705,2
Тамерлан	1 704,0
Цыгане	1 703,9
Геринг, Герман	1 702,3
Джон Уик	1 698,2
Куба	1 697,9
Легион «Свобода России»	1 697,9
Су-25	1 697,6
Монако	1 697,0
Оскар (кинопремия, 2023)	1 696,7
Человек-паук: Паутина вселенных	1 695,6
Климова, Екатерина Александровна	1 694,2
Список столиц государств	1 693,7
Уокер, Пол	1 692,9
Клеопатра	1 692,5
Ас-саляму алейкум	1 692,5
Сбербанк России	1 692,5
Банди, Тед	1 689,0
Крымский мост	1 688,3
Немцов, Борис Ефимович	1 687,9
Мобилизация в России (с 2022)	1 684,8
Властелин колец (кинотрилогия)	1 684,0
Форсаж (франшиза)	1 683,3
Паническая атака	1 682,7
Дания	1 679,9
Оскар (кинопремия)	1 679,6
Макгрегор, Конор	1 677,9
Вавилон (фильм, 2022)	1 676,3
Фитин, Павел Михайлович	1 676,0
Янукович, Виктор Фёдорович	1 674,6
Чеченская Республика Ичкерия	1 674,6
Латвия	1 671,5
Кравец, Марина Леонидовна	1 670,6
Литва	1 670,5
Волга	1 669,9
Категории годности к военной службе	1 669,6
MiyaGi & Andy Panda	1 667,1
Скабеева, Ольга Владимировна	1 667,0
Воронеж	1 665,5
Операция «Фортуна»: Искусство побеждать	1 664,2
Нигерия	1 663,5
Список фильмов о Джеймсе Бонде	1 661,5
Индекс человеческого развития	1 661,0
МакSим	1 660,9
Невзоров, Александр Глебович	1 660,4
Бесстыжие	1 660,1
Агностицизм	1 660,1
Собянин, Сергей Семёнович	1 658,4
Билан, Дима Николаевич	1 657,0
Yung Trappa	1 656,3
Ткачук, Евгений Валерьевич	1 655,2
Рузвельт, Франклин Делано	1 654,5
Континент	1 653,8
Иван III Васильевич	1 653,4
Карпов, Анатолий Евгеньевич	1 651,8
Флэш (фильм)	1 651,5
Расстрел царской семьи	1 651,0
Якутия	1 649,3
Кавилл, Генри	1 647,8
Михалков, Сергей Владимирович	1 647,6
Обама, Барак	1 647,4
Эстония	1 646,8
Туберкулёз	1 645,4
Дауни, Роберт (младший)	1 645,3
Терешкова, Валентина Владимировна	1 645,0
Валовой внутренний продукт	1 645,0
Тяп-ляп	1 644,7
M2 «Брэдли»	1 644,7
Фокс, Меган	1 644,0
Православие	1 643,4
Чимаев, Хамзат Хизарович	1 643,3
Список султанов Османской империи	1 642,6
Сочи	1 641,7
Красовская, Анастасия Олеговна	1 641,5
Андропов, Юрий Владимирович	1 641,4
Пачино, Аль	1 640,9
Эльбрус	1 639,7
Африка	1 639,5
Йоханссон, Скарлетт	1 636,8
Бекхэм, Дэвид	1 636,5
Мэнсон, Чарльз	1 635,0
Уильямс, Робин	1 635,0
BlackRock	1 633,3
Президентские выборы на Украине (2024)	1 633,1
Чемпионат мира по футболу 2022	1 633,0
Тетрис	1 632,5
Алексей Михайлович	1 632,4
Елизавета I	1 631,1
Протестантизм	1 630,9
Понасенков, Евгений Николаевич	1 630,8
Ходченкова, Светлана Викторовна	1 630,1
Леди Гага	1 630,0
Ататюрк, Мустафа Кемаль	1 629,8
Бакальчук, Татьяна Владимировна	1 629,4
Отмена крепостного права в России	1 626,4
Ахеджакова, Лия Меджидовна	1 626,1
Пермь	1 625,7
Украинская Народная Республика	1 623,9
MGM-140 ATACMS	1 623,5
Ацетилсалициловая кислота	1 622,6
Мьянма	1 621,5
Борис Годунов	1 621,5
Уильям, принц Уэльский	1 621,3
Assassin’s Creed	1 620,9
Космодемьянская, Зоя Анатольевна	1 619,8
Тюмень	1 619,0
Oxxxymiron	1 619,0
Град (РСЗО)	1 618,9
Мадонна (певица)	1 618,3
Ахматова, Анна Андреевна	1 616,7
Мария-Антуанетта	1 612,8
Борис Акунин	1 612,6
Украинский язык	1 611,1
Будённый, Семён Михайлович	1 609,6
Челленджер 2	1 609,2
Уолберг, Марк	1 608,6
Роналдо	1 608,0
Чёрно-жёлто-белый флаг	1 607,9
Александрова, Юлия Игоревна	1 607,9
Джонс, Джон (боец)	1 607,2
Московский Кремль	1 606,1
Никулин, Юрий Владимирович	1 605,7
Груз 200 (фильм)	1 603,9
Луна	1 603,8
Хокинг, Стивен	1 603,4
Организация Договора о коллективной безопасности	1 602,0
Ламборгини, Ферруччо	1 601,2
Лето, Джаред	1 600,7
Пажитнов, Алексей Леонидович	1 599,8
Свифт, Тейлор	1 599,1
One Piece. Большой куш	1 599,1
Южная Осетия	1 598,9
Хэнкс, Том	1 596,7
Анна Иоанновна	1 596,3
Хайек, Сальма	1 596,1
Катар	1 595,5
Зенит (футбольный клуб, Санкт-Петербург)	1 595,2
Бегущий по лезвию 2049	1 595,1
Австрия	1 594,1
Бельгия	1 594,0
Кадыров, Ахмат Абдулхамидович	1 593,4
Золотой мяч (France Football)	1 593,3
Список президентов России	1 592,6
Интернет	1 592,1
Чернобыль	1 591,8
Пирамида потребностей по Маслоу	1 591,5
Бисексуальность	1 590,3
Бунин, Иван Алексеевич	1 589,4
Бурдж-Халифа	1 589,3
Цветаева, Марина Ивановна	1 588,9
Григорианский календарь	1 588,5
Холланд, Том (актёр)	1 588,4
Сэндлер, Адам	1 587,0
Греция	1 585,3
Список стран по индексу человеческого развития	1 584,7
Гурченко, Людмила Марковна	1 583,5
Форд, Харрисон	1 583,4
Спецоперация (эвфемизм)	1 583,2
Отчаянные домохозяйки	1 582,1
Этанол	1 580,8
Басилашвили, Олег Валерианович	1 580,8
Чемпионат Европы по футболу 2024 (отборочный турнир)	1 579,1
Секс в большом городе	1 576,8
Миротворец (сайт)	1 576,4
Бондарчук, Фёдор Сергеевич	1 576,3
Болезнь Лайма	1 575,0
Оральный секс	1 573,9
Потери во Второй мировой войне	1 573,4
Саакашвили, Михаил Николаевич	1 572,9
Эритрея	1 572,8
Бумага, Влад	1 572,6
Навка, Татьяна Александровна	1 570,6
Умеров, Рустем Энверович	1 570,2
Крёстный отец (фильм)	1 570,1
Конкорд (компания)	1 569,8
Георг V	1 568,5
Геракл	1 568,3
Пётр II	1 567,9
Геббельс, Йозеф	1 567,1
Путина, Людмила Александровна	1 566,8
Шаламе, Тимоти	1 565,6
Х-101	1 565,2
Тихонов, Вячеслав Васильевич	1 564,5
День народного единства	1 563,6
Baldur’s Gate III	1 561,1
Эйфелева башня	1 560,9
Деревянко, Павел Юрьевич	1 559,3
Лучше звоните Солу	1 559,1
Порнография	1 558,9
Утяшева, Ляйсан Альбертовна	1 558,2
Калкин, Маколей	1 558,1
Мизулина, Елена Борисовна	1 551,0
Серебряков, Алексей Валерьевич	1 550,6
Гедонизм	1 549,3
Спартак (футбольный клуб, Москва)	1 549,1
Маша и Медведь	1 549,0
Людовик XIV	1 548,7
Государство	1 547,9
Орден Трудового Красного Знамени	1 547,8
Карнавал, Валя	1 547,7
Сифилис	1 547,3
Эффект Даннинга — Крюгера	1 547,1
Солженицын, Александр Исаевич	1 545,6
Jägermeister	1 544,8
Новая Зеландия	1 544,7
Need for Speed	1 543,9
Исландия	1 543,4
Албания	1 542,8
Коммунизм	1 542,8
Гимн России	1 540,7
Орден Почёта (Россия)	1 540,1
Ми-8	1 539,9
Список штатов и территорий США по численности населения	1 536,8
'''

In [ ]:
import re

pattern = re.compile(r'[\t\s]+\d[\d\s]*,\d+$')

with open('c.txt', 'w', encoding='utf-8') as f:
  for line in s.split('\n'):
      cleaned_line = pattern.sub('', line)
      f.write(cleaned_line)
      f.write('\n')

In [ ]:
import json
import os

In [ ]:
files = [f for f in os.listdir() if not os.path.isdir(os.path.join(f))]

In [ ]:
all_annotations = []
tmp_annotation = []
for file in files:
  with open(file, 'r', encoding='utf-8') as f:
    tmp_annotation = json.load(f)
    all_annotations.append(tmp_annotation)

In [ ]:
all_annotations = sorted(all_annotations, key=lambda x: x['title'])

In [ ]:
new_ann = [{'title': elem['title'], 'author': elem['author'], 'annotation': elem['annotation'], 'categories': elem['categories']} for elem in all_annotations]

In [ ]:
with open('MEGA_JSON.json', 'w') as f:
  json.dump(new_ann, f, indent=4, ensure_ascii=False)

In [ ]:
titles = [elem['title'] for elem in all_annotations]

In [ ]:
print(titles)

['100 вёрст по реке', '100 лет тому вперёд', '100 тысяч', '101', '11 минут', '11/22/63', '13-й подвиг Геракла', '1408', '16 процентов', '1793', '1922', '1984', '1Q84', '20 тысяч льё под водой', '2062', '30 лет спустя', '35 кило надежды', '39 ступеней', '45 татуировок менеджера', '50 дней до моего самоубийства', '50 оттенков серого', '500 миллионов бегумы', '75 тысяч', '9/10 судьбы', 'Allez!', 'Cвеча', 'En la noche', 'Euthymia', 'Generation «П»', 'P. S. Я тебя люблю', 'RUR', 'SNUFF', 'SOS', 'Salta pro nobis', 'Terra incognita', 'The Тёлки', 'Videoты', 'Vita Nostra', '«Архимед» Вовки Грушина', '«Кадиллак» Долана', '«Капитал» Карла Маркса', 'Ёлка', 'Ёлка с сюрпризом', 'А Воробьёв стекло не выбивал', 'А есть А', 'А тем временем где-то', 'А теперь не смотри', 'Абрек', 'Авессалом, Авессалом!', 'Автоматический тигр', 'Автостопом по галактике', 'Агамемнон и сын его Орест', 'Агафья', 'Агенты', 'Агнес Грей', 'Ад', 'Административная грация', 'Ажар', 'Азазель', 'Аклима', 'Актриса Фостен', 'Акулья 

In [ ]:
pip install sentence_transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 105.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 85.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 45.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [ ]:
pip install datasets zstandard jsonlines pysimdjson

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 44.0 MB/s eta 0:00:00


In [ ]:
pip install -U datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 15.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.


In [ ]:
from sentence_transformers import SentenceTransformer
from datasets import load_dataset

In [ ]:
encoder = SentenceTransformer('deepvk/USER-bge-m3')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
def similarity(a: str, b: str) -> float:
    emb_1 = encoder.encode(a)
    emb_2 = encoder.encode(b)

    return round(float(encoder.similarity(emb_1, emb_2).item()), 3)

In [ ]:
dataset = load_dataset('IlyaGusev/librusec_full', split='train', streaming=True)

README.md:   0%|          | 0.00/1.83k [00:00<?, ?B/s]

librusec_full.py:   0%|          | 0.00/4.84k [00:00<?, ?B/s]

The repository for IlyaGusev/librusec_full contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/IlyaGusev/librusec_full.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] н
The repository for IlyaGusev/librusec_full contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/IlyaGusev/librusec_full.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


In [ ]:
needed_titles = ['100 вёрст по реке', '100 лет тому вперёд', '100 тысяч', '101', '11 минут', '11/22/63', '13-й подвиг Геракла', '1408', '16 процентов', '1793', '1922', '1984', '1Q84', '20 тысяч льё под водой', '2062', '30 лет спустя', '35 кило надежды', '39 ступеней', '45 татуировок менеджера', '50 дней до моего самоубийства', '50 оттенков серого', '500 миллионов бегумы', '75 тысяч', '9/10 судьбы', 'Allez!', 'Cвеча', 'En la noche', 'Euthymia', 'Generation «П»', 'P. S. Я тебя люблю', 'RUR', 'SNUFF', 'SOS', 'Salta pro nobis', 'Terra incognita', 'The Тёлки', 'Videoты', 'Vita Nostra', '«Архимед» Вовки Грушина', '«Кадиллак» Долана', '«Капитал» Карла Маркса', 'Ёлка', 'Ёлка с сюрпризом', 'А Воробьёв стекло не выбивал', 'А есть А', 'А тем временем где-то', 'А теперь не смотри', 'Абрек', 'Авессалом, Авессалом!', 'Автоматический тигр', 'Автостопом по галактике', 'Агамемнон и сын его Орест', 'Агафья', 'Агенты', 'Агнес Грей', 'Ад', 'Административная грация', 'Ажар', 'Азазель', 'Аклима', 'Актриса Фостен', 'Акулья клетка', 'Али и Нино', 'Алина и Авдей', 'Алмаз Раджи', 'Алое кольцо', 'Алые паруса', 'Алый первоцвет', 'Аль-Исса', 'Альбом', 'Альтист Данилов', 'Алюминиевое солнце', 'Алёнка', 'Алёша Горшок', 'Алёша Птицын вырабатывает характер', 'Алёшкино сердце', 'Аман и его дочери', 'Америка', 'Американские партизаны', 'Амок', 'Амстердам', 'Анафема', 'Ангел-хранитель', 'Андеграунд', 'Андрокл и лев', 'Антигона', 'Антихрист', 'Антон Павлович Чехов', 'Антоновские яблоки', 'Анюта', 'Апрельское колдовство', 'Арап Петра Великого', 'Арбузная корка', 'Аргонавты', 'Ариэль', 'Ария Каварадосси', 'Арктур — гончий пёс', 'Арно', 'Арсена Гийо', 'Артёмка', 'Артёмка в цирке', 'Артёмка и гимназисты', 'Архив Шерлока Холмса', 'Арчибал', 'Аскыр', 'Астронавты', 'Ася', 'Атлант расправил плечи', 'Аттестат зрелости', 'Аугсбургский меловой круг', 'Аутодафе', 'Ах, зубы!', 'Ашик-Кериб', 'Аэлита', 'Аякс', 'Бабуля', 'Бабушка', 'Бабушкин подарок', 'Бабье лето Джонсона Сухого Лога', 'Бабьи души', 'Баваклава', 'Бавкида', 'Багира', 'Багульник', 'Баламутка', 'Балкон', 'Баллада', 'Бамбус', 'Баня', 'Баран-непомнящий', 'Баранка — хлеб насущный', 'Баранкин, будь человеком!', 'Барбос и Жулька', 'Баргамот и Гараська', 'Барышня Клара', 'Барышня-крестьянка', 'Батраки', 'Баттен, Баттен!', 'Батый', 'Бахман', 'Бахчевник', 'Башня Волверден', 'Башня ласточки', 'Баязет', 'Беглец', 'Беглые встречи', 'Бегство от свободы', 'Бегущий в лабиринте', 'Бегущий за ветром', 'Беда', 'Бедная Лиза', 'Бедный волк', 'Бежин луг', 'Без заглавия', 'Без противоречий', 'Без улыбок', 'Безбожный переулок', 'Бездна', 'Беззащитное существо', 'Безлюбый', 'Безмолвный свитедель', 'Безобразная герцогиня', 'Безотцовщина', 'Белая дорога', 'Белая кошка', 'Белая птица', 'Белая цапля', 'Белогрудка', 'Белое безмолвие', 'Белолобый', 'Белые одежды', 'Белые слоны', 'Белый квадрат', 'Белый пудель', 'Бельмесов', 'Беляночка и Розочка', 'Бемби', 'Бен-Гур', 'Берег удачи', 'Береника', 'Берилловая диадема', 'Берл-Айзик', 'Бесплатно', 'Бесплодные усилия любви', 'Бесприданница', 'Бессмертные книги', 'Бешеные деньги', 'Бизнес в позе лотоса', 'Бизнес в стиле Ж', 'Бизнес-приключения', 'Бирюк', 'Благородный жулик', 'Благость', 'Блаженный', 'Бледная Анна', 'Блистающий мир', 'Блуждающие звёзды', 'Бляха полицейского О’Руна', 'Бобок', 'Бог как иллюзия', 'Бог мелочей', 'Богатство', 'Богатство народов', 'Богатырь', 'Боги', 'Божье дерево', 'Божья Матерь в кровавых снегах', 'Бой', 'Бойцовский клуб', 'Болтик', 'Боль — это друг, но не всегда со знаком «плюс»', 'Больная', 'Большая четвёрка', 'Больше, чем бизнес', 'Большой Мольн', 'Большой шлем', 'Бондаровна', 'Бородино', 'Борьба за огонь', 'Ботинки посла', 'Бочка', 'Брат, которому семь', 'Братец и сестрица', 'Братоубийство', 'Братья', 'Братья Львиное сердце', 'Братья разбойники', 'Браун из Калавераса', 'Бременские музыканты', 'Бригадир', 'Бродяги Севера', 'Брошечка', 'Бубен Верхнего мира', 'Будет ласковый дождь', 'Будущее воспоминание', 'Будь лучшей версией себя', 'Будь я Ротшильдом', 'Будьте готовы, Ваше высочество', 'Буколики', 'Бульба', 'Бумажные города', 'Бумажный змей', 'Бунт невесток', 'Бурмистр', 'Буянка', 'Бывают минуты', 'Был настоящим трубачом', 'Были они смуглые и золотоглазые', 'Былое и думы', 'Быт и нравы пушкинской эпохи', 'Быть бы свадьбе, да музыки не нашлось', 'Быть начальником — это нормально', 'Бэзил', 'Бэла', 'Бюг-Жаргаль', 'В Аркадии проездом', 'В Кохании', 'В Париже', 'В аптеке', 'В бескрайнем небе', 'В воскресенье рано зелье собирала', 'В глубине памяти', 'В городском саду', 'В гостинице «Наряд Арлекина»', 'В день свадьбы', 'В деревне', 'В добрый час', 'В дороге', 'В дурном обществе', 'В зеркале', 'В исправительной колонии', 'В книгах — зло', 'В краю лесов', 'В лесу', 'В людях', 'В маленьком мире маленьких людей', 'В ночном море', 'В одной знакомой улице', 'В полесской глуши', 'В потопе времени', 'В прекрасном и яростном мире', 'В служении Отечеству', 'В списках не значился', 'В старом доме', 'В степи', 'В сурдокамере', 'В те последние мгновения', 'В тридевятом царстве', 'В школу', 'В этом году я…', 'Ваганов', 'Важные годы', 'Вакханки', 'Валькины друзья и паруса', 'Валя', 'Вам и не снилось', 'Вам привет от бабы Леры', 'Вампир в Суссексе', 'Варвара Ивановна', 'Варька', 'Василий Шибанов', 'Васюткино озеро', 'Васёк Трубачёв и его товарищи', 'Вау-проекты', 'Вверх по лестнице, ведущей вниз', 'Вверху справа — солнце', 'Вглядеться в прошлое, понять будущее', 'Вдали от обезумевшей толпы', 'Ведите себя прилично, Арчибальд!', 'Ведьма с Портобелло', 'Ведьмы', 'Веер', 'Везенье дурака', 'Везир Ленкоранского ханства', 'Великие по собственному выбору', 'Великие путешественники', 'Великий переполох среди маленьких людей', 'Великое противостояние', 'Вельд', 'Вендетта', 'Верблюжий глаз', 'Вересковый мёд', 'Верный Трезор', 'Верный друг Санчо', 'Вероника', 'Вероника решает умереть', 'Весенние перевёртыши', 'Весна порционно', 'Весной, в Иудее', 'Весёлая компания', 'Весёлая семейка', 'Весёлое утро', 'Весёлый двор', 'Ветер в ивах', 'Вечер', 'Вечер на бивуаке', 'Вечер накануне Ивана Купала', 'Вечера на хуторе близ Диканьки', 'Вечная тьма', 'Вечно живые', 'Вечный зов', 'Вечный хлеб', 'Вешние воды', 'Взыскание погибших', 'Визитные карточки', 'Виктор Поликарпович', 'Вильям Вильсон', 'Виннипегский волк', 'Вино из одуванчиков', 'Виноваты звёзды', 'Виноград', 'Витя Малеев в школе и дома', 'Вишнёвые деревца', 'Владычица озера', 'Властелин мира', 'Властелин молний', 'Власть от Бога, но власть — не Бог', 'Во глубине России', 'Во что я верю', 'Военная тайна', 'Вождь краснокожих', 'Возвращение', 'Возвращение Казановы', 'Возвращение Чорба', 'Возвращение Шерлока Холмса', 'Возвращение в детство', 'Возвращение на родину', 'Возвращённая молодость', 'Возница', 'Возрасты жизни', 'Война и мир', 'Война у Титова пруда', 'Вокруг света за 80 дней', 'Волки', 'Волки и овцы', 'Володя', 'Волшебная палочка Аксакова', 'Волшебная шкатулка', 'Волшебник Земноморья', 'Волшебник Изумрудного города', 'Волшебное кольцо', 'Волшебные чары луны', 'Волшебный профиль', 'Вольга и Микула Селянинович', 'Воплощение идей', 'Ворон', 'Ворон-челобитчик', 'Ворота в небо', 'Воры', 'Восковая персона', 'Воспламеняющая взглядом', 'Воспоминания', 'Восстание', 'Восходящее солнце', 'Вот бежит собака', 'Вот и лето прошло', 'Вот как это было!', 'Враги', 'Враньё', 'Врата судьбы', 'Время вокзала', 'Время всегда хорошее', 'Время жить и время умирать', 'Всадник, скачущий над городом', 'Все грехи мира', 'Все истории о маленьких женщинах и маленьких мужчинах', 'Все по местам!', 'Все цвета радуги', 'Всемогущий текст-процессор', 'Всех ожидает одна ночь', 'Встретились, поговорили', 'Встреча', 'Встреча в Багдаде', 'Встреча с собой', 'Всё лето в один день', 'Вторая модель', 'Второе дыхание', 'Второе пятно', 'Второй кофейник', 'Второй май после Октября', 'Вход в рай', 'Вчера', 'Вы подождёте?', 'Выбор', 'Выбор невесты', 'Выборы', 'Выигрышный билет', 'Выкуп', 'Высотка', 'Выстрел', 'Выше Радуги', 'Вышедший из моря', 'Вьюга', 'Вяленая вобла', 'Габриэла, корица и гвоздика', 'Гадкие лебеди', 'Гадюка', 'Галльская кровь', 'Галоша', 'Галоши и мороженое', 'Галстучник', 'Галя Ганская', 'Гамбринус', 'Гамлет Щигровского уезда', 'Гармония в природе', 'Гарри Поттер', 'Гарри Поттер и Дары смерти', 'Гарри Поттер и Кубок огня', 'Гарри Поттер и Орден Феникса', 'Гарри Поттер и Принц-полукровка', 'Гарри Поттер и Проклятое дитя', 'Гарри Поттер и Тайная комната', 'Гарри Поттер и узник Азкабана', 'Гарри Поттер и философский камень', 'Где колбаса и чара, там кончается свара', 'Гедда Габлер', 'Гектор Сервадак', 'Гензель и Гретель', 'Гениальность на заказ', 'Гений', 'Генрих', 'Генрих V', 'Географ глобус пропил', 'Геракл', 'Герберт Уэст — реаниматор', 'Герои', 'Герой нашего времени', 'Гецл', 'Гибель Дергунова', 'Гибкое сознание', 'Гиена', 'Гимназия', 'Гитл Пуришкевич', 'Главные реки Америки', 'Глория Скотт', 'Глупость мертвеца', 'Гном', 'Гнусный обманщик', 'Говорящий свёрток', 'Год синей крысы', 'Голодные игры', 'Голодный город', 'Голос', 'Голос большого города', 'Голос в темноте', 'Голубая звезда', 'Голубая комната', 'Голубая стрекоза', 'Голубая чашка', 'Голубое и зелёное', 'Голубой дым', 'Голубой карбункул', 'Голубой период де Домье-Смита', 'Голубой свет', 'Голубые зоны', 'Голубые открытки', 'Голубь', 'Голый среди волков', 'Гонимые', 'Гонители', 'Гончая смерти', 'Горбун', 'Гордячка', 'Горе', 'Горе одному', 'Горе от ума', 'Горжусь', 'Город Мастеров', 'Город героев', 'Город маленьких людей', 'Городок в табакерке', 'Горшок', 'Горшочек, не вари!', 'Горя бояться — счастья не видать', 'Горячий камень', 'Горящий светильник', 'Господин из Сан-Франциско', 'Госпожа Метелица', 'Гость', 'Государство', 'Государство будущего', 'Государство и революция', 'Грамматика любви', 'Гранат', 'Гранатовый браслет', 'Граф Калиостро', 'Граф Нулин', 'Графиня Рудольштадт', 'Графиня де Монсоро', 'Гриша', 'Гробница Императора', 'Гробницы Атуана', 'Гробовщик', 'Гроза', 'Грозный Мешпалван', 'Громила', 'Грузовики', 'Губастый', 'Губернатор', 'Губительно приятные голоса', 'Гувернантка', 'Гулящая', 'Гулящие люди', 'Гум-Гам', 'Гуси', 'Гуси-лебеди', 'Гуси-лебеди летят', 'Давай здесь останемся жить', 'Дагон', 'Дальнейшие приключения Робинзона Крузо', 'Далёкое', 'Дамское счастье', 'Даниэль Штайн, переводчик', 'Дары волхвов', 'Дашенька', 'Дающий', 'Даёшь сердце', 'Два актёра на одну роль', 'Два антисемита', 'Два года каникул', 'Два клёна', 'Два мертвеца', 'Два пистолета, один капкан', 'Два помещика', 'Два приятеля', 'Два сапога — пара', 'Два слова', 'Два штрафных очка', 'Две легенды', 'Две судьбы', 'Двенадцатая койка', 'Двойник', 'Двойной орешек', 'Двухмужняя', 'Двухсотлетний человек', 'Дебют Мэгги', 'Дева Малейн', 'Дева в беде', 'Девица Эльза', 'Девочка выздоровела', 'Девочка из города', 'Девочка ищет отца', 'Девочка на урезе моря', 'Девочка на шаре', 'Девочка с Земли', 'Девочка со спичками', 'Девочка, наступившая на хлеб', 'Девочка, с которой ничего не случится', 'Девушка online', 'Девушка с кувшином', 'Девушка-одуванчик', 'Девятный Спас', 'Дед Мазай и зайцы', 'Деда', 'Дедушка', 'Дедушка Сергали', 'Дедушкин отель', 'Дела земные', 'Дело «пёстрых»', 'Дело госпожи Люно', 'Дело о мёртребе', 'Дело о пропавшем носе', 'Дело пропавшей леди', 'Дело розовой жемчужины', 'Делёж кабанчика', 'Демагоги', 'Демон', 'Демоны в раю', 'Денискины рассказы', 'День гнева', 'День опричника', 'День рождения', 'День рождения Алисы', 'День рождения Инфанты', 'День с Германом Титовым', 'День сурка', 'День триффидов', 'День, когда рухнул мир', 'Деньги', 'Деньги для Марии', 'Деревенский пожар', 'Деревня', 'Деревья растут для всех', 'Деревянная пастушка', 'Державин', 'Десантно-штурмовая бригада', 'Десять негритят', 'Детвора', 'Дети', 'Дети Онегина и Татьяны', 'Дети капитана Гранта', 'Дети колонии «Жижа»', 'Дети кочегара', 'Дети кукурузы', 'Дети мои', 'Дети на стройке', 'Дети полуночи', 'Дети синего фламинго', 'Дети солнца', 'Детство', 'Детство Никиты', 'Детство в Архангельске', 'Джельсомино в Стране лжецов', 'Дженишбек', 'Джентльмен из Лапорта', 'Джефф Питерс как персональный магнит', 'Джим с Пиккадилли', 'Джонни-мнемоник', 'Джуд Незаметный', 'Дзяды', 'Диалог на литературно-художественные темы', 'Дивергент', 'Дизайн привычных вещей', 'Дикая охота короля Стаха', 'Дикий голубь', 'Дикий гусь', 'Дикий помещик', 'Динка', 'Динка прощается с детством', 'Дипломат', 'Дитя Марии', 'Дитё', 'Для увеселенья', 'Дневник Коли Синицына', 'Дневник памяти', 'До Адама', 'До встречи с тобой', 'Добрая книга — как всхожее зерно', 'Добрая подружка', 'Добродетели и Пороки', 'Доброта', 'Добыча', 'Доверие', 'Договориться можно обо всём', 'Дождь', 'Дожить до рассвета', 'Доктор Греслер', 'Доктор Крупов', 'Доктора!', 'Долгие разговоры о жизни', 'Долина ужаса', 'Дом в тысячу этажей', 'Дом кошки, играющей в мяч', 'Дом на дюнах', 'Дом с характером', 'Дом, в котором…', 'Домашние роды', 'Домик на юге', 'Домино', 'Домой на Пасху', 'Дон Кихот', 'Дон Кихот и донкихотство', 'Донские рассказы', 'Дороги судьбы', 'Дороги, которые мы выбираем', 'Дорогие мои мальчишки', 'Дорогой ценой', 'Дорожное происшествие', 'Доставляя счастье', 'Достоинство', 'Дочь Альбиона', 'Дочь Лилит', 'Дочь Монтесумы', 'Дочь Раппаччини', 'Дочь моя, Анюта', 'Дочь священника', 'Драка', 'Дракула', 'Драма в Лифляндии', 'Драма на охоте', 'Древнерусская литература и современность', 'Дрейфус в Касриловке', 'Друг Телемак', 'Друг детства', 'Дружба', 'Дружба кошки и мышки', 'Дубки', 'Дубровский', 'Думы', 'Дурак', 'Дурак в поход собрался', 'Дурачок', 'Дурочка', 'Духless', 'Душа крупье', 'Дым в лесу', 'Дьявол', 'Дьяволова нога', 'Дюймовочка', 'Дюна', 'Дядя Ермолай', 'Дядя Жюль', 'Дядя Фёдор, пёс и кот', 'Дядя Хлор и Корякин', 'Евгений Онегин', 'Евгений и Юлия', 'Еврейская бригада', 'Егерь', 'Его батальон', 'Его прощальный поклон', 'Егор Булычов и другие', 'Ей во вред живущая', 'Елтышевы', 'Ель', 'Емшан', 'Ермолай и мельничиха', 'Если кто меня слышит', 'Есть, молиться, любить', 'Жалобная книга', 'Жалость ко всему живому', 'Жангада', 'Жарким летом', 'Жгучая тайна', 'Железная Шерсть', 'Железная дорога', 'Железная пята', 'Железнодорожные рассказы', 'Железное алиби', 'Женитьба Бальзаминова', 'Женщина в пословицах и поговорках народов мира', 'Женщины Лазаря', 'Женщины в народном собрании', 'Жеребёнок', 'Жестокий век', 'Живи и дай умереть', 'Живое пламя', 'Живые мощи', 'Живёт такой парень', 'Жизнь Александра Грина', 'Жизнь Гнора', 'Жизнь Трезора', 'Жизнь вечна', 'Жизнь взаймы', 'Жизнь за царя', 'Жизнь и воротник', 'Жизнь и удивительные приключения Робинзона Крузо', 'Жизнь как конструктор', 'Жизнь как стартап', 'Жилища богатырей', 'Житейские истории', 'Жозеф Фуше', 'Журавлиный крик', 'Жёлтое лицо', 'Жёлтый Туман', 'Жёлтый князь', 'Жёны и дочери', 'Жёсткая проба', 'За всё хорошее — смерть', 'За дверью', 'За двумя зайцами', 'За миллиард лет до конца света', 'За нами Москва', 'За права', 'За сестрой', 'За советом', 'Заблудившийся автобус', 'Заброшенная обсерватория', 'Забытые мечты', 'Забытый мир', 'Завещание', 'Завтра была война', 'Загадка', 'Загадка Богдановича', 'Загадка Н. Ф. И.', 'Загадка Ситтафорда', 'Загадка Торского моста', 'Загадка поместья Шоскомб', 'Загадки сна', 'Закат Европы', 'Закатное солнце', 'Заколдованная буква', 'Заколдованный замок', 'Заколдованный портной', 'Заколдованный спектакль', 'Закон и жена', 'Закрой последнюю дверь', 'Заложники Момуса', 'Заметки о русском', 'Заметы духовные', 'Замок Отранто', 'Замок в Карпатах', 'Занавес', 'Занимательная Греция', 'Занятой человек', 'Запах мысли', 'Записки из Мёртвого дома', 'Записки о Шерлоке Холмсе', 'Записки охотника', 'Записки сумасшедшего', 'Записки юного врача', 'Запретное царство', 'Запретные цвета', 'Заря всю ночь', 'Застольное слово о Пушкине', 'Затерянный мир', 'Затишье', 'Затянувшийся дебют Лоис Тэггетт', 'Захар Беркут', 'Заххок', 'Зачем я убил коростеля?', 'Заячьи лапы', 'Звезда КЭЦ', 'Звезда Севильи', 'Звездопад', 'Звезды Эгера', 'Звоните и приезжайте', 'Звонок', 'Звёздная сыпь', 'Звёздные корабли', 'Звёздные талеры', 'Звёздный десант', 'Звёздный скиталец', 'Звёзды', 'Звёзды под дождём', 'Звёзды смотрят вниз', 'Здесь тоже водятся тигры', 'Здравомысленный заяц', 'Зелень к празднику', 'Зелёная дверь', 'Зелёная лампа', 'Зелёная миля', 'Зелёная ночь', 'Зелёное утро', 'Земля Санникова', 'Земля под её ногами', 'Земля родная', 'Земляки', 'Земляничная весна', 'Земляничное окошко', 'Земное эхо солнечных бурь', 'Зимний дуб', 'Зло под солнцем', 'Злой мальчик', 'Злоумышленник', 'Злоумышленники', 'Злым ветром', 'Змея в кулаке', 'Знак вечности', 'Знак небесный', 'Знатный холостяк', 'Знахарь', 'Зов Ктулху', 'Зов крыльев', 'Зов предков', 'Зойка и Валерия', 'Золотая карета', 'Золотая роза', 'Золотая цепь', 'Золото бунта', 'Золото и любовь', 'Золотое дно', 'Золотое пенсне', 'Золотой гусь', 'Золотой компас', 'Золотой мальчик', 'Золотой петух', 'Золотопряды', 'Золотые слова', 'Золотые туфельки', 'Золушка', 'Зулейха открывает глаза', 'И грянул гром', 'И каждый день — сначала', 'И на всю жизнь дар ученичества', 'И пришёл бука', 'И эхо летит по горам', 'Иван Бунин', 'Иван Сусанин', 'Иван Фёдорович Шпонька и его тётушка', 'Иван — крестьянский сын и Чудо-юдо', 'Иванов и Рабинович', 'Игорь-Робинзон', 'Игра', 'Игра Эндера', 'Игра в имитацию', 'Игра в любовь', 'Игрушечного дела людишки', 'Игрушка', 'Игры, в которые играют люди', 'Ида', 'Из Донгри в Дубай', 'Из глубины', 'Из жизни микробов', 'Из записных книжек', 'Из записок об А. С. Грине', 'Из любви к искусству', 'Из тьмы веков', 'Из-за редьки', 'Из-под дождя да под ливень', 'Избавилась', 'Избушка в лесу', 'Избыток природы', 'Изгнание', 'Изгнанники Покер-Флэта', 'Изгои', 'Изумруд', 'Изыди, Сатана', 'Или — или', 'Иллюзион', 'Иллюзионист Эйзенхайм', 'Ильинский омут', 'Илья Муромец', 'Илья Муромец и Соловей-разбойник', 'Илья и Соловей', 'Илюха', 'Иммензее', 'Инкубатор на столе', 'Иногда они возвращаются', 'Инородное тело', 'Институтка', 'Инструменты McKinsey', 'Интеграция', 'Интервенция', 'Инферно', 'Инь и Ян', 'Иоанн Рыдалец', 'Иов', 'Ионыч', 'Иоселе-соловей', 'Иосиф', 'Исаак Левитан', 'Искатель приключений', 'Искусный вор', 'Искусство и ковбойский конь', 'Искусство управлять', 'Испанская баллада', 'Испанский английский', 'Исповедь', 'Исповедь «неполноценного» человека', 'Исповедь маски', 'Исповедь на электрическом стуле', 'Исторические портреты', 'Исторический роман', 'Историю не надо предсказывать, её надо понимать', 'История Пугачёва', 'История болезни', 'История греха', 'История жилички под вуалью', 'История крепостного мальчика', 'История лейтенанта Ергунова', 'История об отрубленной руке', 'История одного города', 'История с «зелёным»', 'История с привидением', 'История с тремя городами', 'История твоей жизни', 'Исчезновение Чёрного Орла', 'Исчезновение леди Фрэнсис Карфэкс', 'Йоринда и Йориндель', 'К нему тянулись люди', 'Кавказ', 'Кавказcкий пленник', 'Кавказский пленник', 'Каждый мечтает о собаке', 'Казак-стихотворец', 'Казус Кукоцкого', 'Как быть интересным', 'Как в полдень', 'Как вам это понравится', 'Как вести игру с судьбой', 'Как жаль', 'Как зайка летал на воздушных шариках', 'Как меня звать?', 'Как один мужик двух генералов прокормил', 'Как писать хорошо', 'Как прозрел Доггерти', 'Как решить пять основных проблем команды', 'Как я ловил человечков', 'Как я мечтал о бескорыстии', 'Как я редактировал сельскохозяйственную газету', 'Как я стал писателем', 'Какой случится день недели', 'Кактус', 'Календарь русской природы', 'Калигула', 'Каллисто', 'Каллистяне', 'Калоши', 'Калоши счастья', 'Камарг', 'Каменный гость', 'Каменный крест', 'Каменный цветок', 'Камень Мазарини', 'Каникулы', 'Канитель', 'Капитан Сорви-голова', 'Капитанская дочка', 'Капроновая ёлочка', 'Кара-Бугаз', 'Карась-идеалист', 'Карибская тайна', 'Карлик Нос', 'Карман, полный ржи', 'Кармилла', 'Карниз', 'Картина', 'Картонная коробка', 'Картофельный эльф', 'Карты на стол', 'Картёжник и бретёр, игрок и дуэлянт', 'Карьера Ругонов', 'Касриловская богадельня', 'Касриловские погорельцы', 'Касриловский прогресс', 'Кассандра', 'Кастрюк', 'Касьян с Красивой Мечи', 'Катенька', 'Катя и Чудеса', 'Катя и крокодил', 'Кафка на пляже', 'Качели', 'Каштанка', 'Кедр', 'Кентервильское привидение', 'Кирпичные острова', 'Кисель', 'Клад', 'Кладбище домашних животных', 'Кладбищенский фантом', 'Кладовая солнца', 'Класс коррекции', 'Клацающие зубы', 'Клик', 'Климко', 'Клод Гё', 'Клоун Шалимар', 'Клошет', 'Клуб самоубийц', 'Книга как врата в рай', 'Книгоноши', 'Книжный вор', 'Кнопка, кнопка', 'Княжна Мери', 'Князь Витовт', 'Князь Михайло Репнин', 'Князь во князьях', 'Когда в доме одиноко', 'Когда вырастет на камушке зелёная трава', 'Когда книга учит', 'Код да Винчи', 'Козёл отпущения', 'Колесницы богов', 'Колобок', 'Коловерть', 'Коловращение', 'Колокольня', 'Колония нескучного режима', 'Колотый сахар', 'Коляска', 'Команда корректировки', 'Комната на чердаке', 'Компромисс девятый', 'Кому на Руси жить хорошо', 'Конец', 'Конец Света', 'Конец Чарльза Огастеса Милвертона', 'Конец Чертопханова', 'Конец всей этой гадости', 'Конкуренты', 'Конотопская ведьма', 'Консалтинг на миллион', 'Контакт', 'Контора', 'Контрабас', 'Конь и его мальчик', 'Конь с розовой гривой', 'Коняга', 'Копи царя Соломона', 'Корабли в Лиссе', 'Коралина', 'Корзина с еловыми шишками', 'Кориолан', 'Корней Васильев', 'Корова', 'Королева красоты Иерусалима', 'Королевская Аналостанка', 'Королевство кривых зеркал', 'Короленко в кругу друзей', 'Короли и капуста', 'Король Дроздобород', 'Король забавляется', 'Король, дама, валет', 'Королёк', 'Королёк — птичка певчая', 'Коротенький отпуск мистера Лавдэя', 'Корпорация «Бросайте курить»', 'Кортик', 'Космос, нервная система и шмат сала', 'Косцы', 'Кот в сапогах', 'Кот из ада', 'Коты-воители', 'Преступление и наказание']

In [ ]:
import json
from tqdm import tqdm

with open('All_annotations.json', 'r', encoding='utf-8') as f:
    all_a = json.load(f)

In [ ]:
print(all_a[0]['author'])

Грин


In [ ]:
annotations_dict = {}

for a in all_a:
    annotations_dict[a['title']] = a

In [ ]:
print(annotations_dict['Детство']['author'])

Науменко


In [ ]:
print(len(annotations_dict))

1135


In [ ]:
SIM_TH = 0.65
texts_dict = {}
needed_titles = set(needed_titles)

with tqdm(total=496858, desc="Processing records") as pbar:
    for record in dataset:
        title = record.get("title", "")
        authors = record.get("authors", [""])
        lang = record.get("lang", "")

        if title in needed_titles:
            if title not in texts_dict:
                if lang in ['ru', 'rus']:
                    if annotations_dict[title]['author'] in authors:
                        texts_dict[title] = (record.get("sections", ""), authors)
                    else:
                        for author in authors:
                            if similarity(author, annotations_dict[title]['author']) > SIM_TH:
                                texts_dict[title] = (record.get("sections", " "), authors)
                                break

        pbar.set_postfix({
            'found': len(texts_dict),
        })
        pbar.update(1)

        if len(texts_dict) == len(needed_titles):
            break

Processing records: 100%|██████████| 496858/496858 [4:05:20<00:00, 33.75it/s, found=634]


In [ ]:
print(len(texts_dict))

634


In [ ]:
combined = []
for title, value in texts_dict.items():
  combined.append({'title': title, 'author': annotations_dict[title]['author'], 'authors': value[1], 'annotation': annotations_dict[title]['annotation'],'text': value[0], 'categories': annotations_dict[title]['categories']})

In [ ]:
combined[0]

{'title': 'Башня Волверден',
 'author': 'Аллен',
 'authors': ['Аллен Грант'],
 'annotation': 'Мистическая история о старинной башне, где чуть не погибла впечатлительная девушка.\n\nДвадцатилетняя дочь антиквара Мэйзи Льюэллин перед Рождеством приглашена погостить в замок елизаветинской эпохи Волверден-холл в окрестностях Лондона. Его хозяйка миссис Уэст встречает девушку с великосветской церемонией, показывает ей замок и её спальную. Девушка поражена роскошью и величием замка, у которого славная история. Сама девушка\xa0— представительница древнего рода, «была высокой и хрупкой, с густыми чёрными волосами и одухотворёнными чертами лица, как оно и подобает потомку Льюэллина ап Йорверта».\n\nВ замок съехалось много гостей посмотреть на представление\xa0— живые картины. Мэйзи никого не знает. Из её окна открывается прекрасный вид на деревенские просторы, старую церковь и новую, перестроенную, белую башню. Мэйзи подмечает контраст старых строений и новодела, что расстраивает хозяйку. Та ра

In [ ]:
with open('combined_data.json', 'w', encoding='utf-8') as f:
  json.dump(combined, f, indent=4, ensure_ascii=False)